In [1]:
import numpy as np
import pandas as pd
import pickle

import FinanceDataReader as fdr
from pykrx import stock
from tqdm import tqdm

from numba import njit, jit
import warnings
warnings.filterwarnings('ignore')

In [2]:
start_date = "2013-04-01"
end_date = "2023-04-01"

In [3]:
df_spx = fdr.StockListing("KOSPI")
tickers = df_spx['Code']

In [4]:
def return_price_cap_pykrx(tickers, start_date, end_date):
    price_df = pd.DataFrame()
    cap_df = pd.DataFrame()
    for ticker in tqdm(tickers):
        temp = stock.get_market_ohlcv(start_date, end_date, ticker)
        temp = temp.astype('float')
        temp_price = temp['종가'].to_frame(ticker)
        temp_cap = (temp['종가']*temp['거래량']).to_frame(ticker)
        price_df = pd.concat([price_df, temp_price],axis=1)
        cap_df = pd.concat([cap_df, temp_cap], axis=1)
    return price_df, cap_df

In [5]:
price_df, cap_df = return_price_cap_pykrx(tickers, start_date, end_date)
price_df = price_df.dropna(axis=1)
cap_df = cap_df.dropna(axis=1)

100%|██████████| 948/948 [02:29<00:00,  6.35it/s]


In [6]:
print(cap_df.columns.tolist() == price_df.columns.tolist())
tickers = price_df.columns.tolist()
print(len(tickers))

True
770


In [7]:
def return_fundamental_data(tickers, start_date, end_date):
    per = pd.DataFrame()
    pbr = pd.DataFrame()
    div = pd.DataFrame()
    for ticker in tqdm(tickers):
        temp = stock.get_market_fundamental(start_date, end_date, ticker)
        temp = temp.astype('float')
        try:
            if (0 in temp['PER'].tolist()) or (0 in temp['PBR'].tolist()) or (0 in temp['DIV'].tolist()):
                pass
            else:
                temp_per = temp['PER'].to_frame(ticker)
                temp_pbr = temp['PBR'].to_frame(ticker)
                temp_div = temp['DIV'].to_frame(ticker)
                per = pd.concat([per,temp_per ], axis=1)
                pbr = pd.concat([pbr, temp_pbr], axis=1)
                div = pd.concat([div, temp_div], axis=1)
        except:
            pass
    return per, pbr, div

In [8]:
per, pbr, div = return_fundamental_data(tickers, start_date, end_date)

100%|██████████| 770/770 [1:01:43<00:00,  4.81s/it]


In [9]:
tickers = per.columns.tolist()
print(len(tickers))

186


In [10]:
krx_dict = {}
krx_dict['price'] = price_df
krx_dict['cap'] = cap_df
krx_dict['per'] = per
krx_dict['pbr'] = pbr
krx_dict['div'] = div

In [11]:
# pickle로 저장하기
with open('krx_dict.pkl','wb') as f:
    pickle.dump(krx_dict, f)